# exporting the Breton example sentences

In [2]:
import datetime, tqdm, re, time
#!pip install mwclient
#!pip install mwparserfromhell
#!pip install mwcomposerfromhell
from mwclient import Site
# import dominate, pypandoc
# from dominate.tags import *
# from dominate.util import raw
#myparserfromhell : parser pour le code wiki
import mwparserfromhell, mwcomposerfromhell

In [ ]:
#!pip install --upgrade ipykernel
#!pip install mwcomposerfromhell
#%pip install pypandoc

### Si on veut charger toutes les pages du site :

In [ ]:
print('connecting...')
#site contenant tous les exemples
site = Site('arbres.iker.cnrs.fr/', path='/')
#site.login('Kimgerdes', 'azerAZER1')
outf = open('problemPages.html','w')

# images = str(list(site.allimages()))
#print(images)
#print(dir(images[0]))
#print(images[0].name)
#qsdf
allpages = list(site.allpages())
print('got all', len(allpages), 'pages')

#### Si on veut sauvegarder les pages chargées dans un fichier Pickle

In [ ]:
#t = time.time()
#dictionnaire contenant toutes les pages aspirées
#pages = {}
#on peut ici modifier le nombre de pages à charger pour charger petit à petit ex : allpages[:3000] pour les 3000 premières pages
#newlist=allpages[:8068]
#for p in tqdm.tqdm(allpages):
#for p in tqdm.tqdm(newlist):
#	#si les pages ne sont pas de la documentation
#	if not('/documentation' in p.name or '/docname' in p.name or 'Module:' in p.name):
#		#on ajoute le nom de la page comme clé, son contenu comme valeur
#		pages[p.name] = p.text() 
#print('extraction done. it took', time.time()-t, 'seconds.')

In [ ]:
#une fois toutes les pages chargées, on le met dans un pickle
#with open("Dict.txt", "wb") as myFile:
#    pickle.dump(pages, myFile)

In [3]:
#on utilise le(s) pickle(s)
import pickle
with open("Dict.txt", "rb") as myFile:
    pages = pickle.load(myFile)

In [4]:
with open("Sources.txt", "rb") as file:
    sources = pickle.load(file)

In [5]:
# print(', '.join(pages.keys()))
#on créé un fichier txt contenant tous les titres de pages
open('pageTitles.txt','w', encoding="utf-8").write('\n'.join(pages.keys()))
print('extracted',len(pages),'pages')

extracted 8068 pages


In [51]:
nbpretty=0
#on parcourt les clés et valeurs du dictionnaire pages
for name,text in pages.items():
	#et on compte le nombre de tableaux (donc d'exemples)
	nbpretty += text.count('class="prettytable"')
print('found',nbpretty,'examples')

found 13422 examples


In [10]:
"""
{| class="prettytable"
|(1)|| N'eo || ket || dleet || koduiñ || hag || evañ || er || memes || amzer.
|-
||| [[ne]] [[COP|est]] || [[ket|pas]] || [[dleout|dû]] || conduire || [[&|et]] || [[evañ|boire]] || [[P.e|en]].[[art|le]] || [[memes|même]] || temps
|-
|||colspan="10" | 'On ne doit pas boire et conduire en même temps /Il ne faut pas boire et conduire' 
|-
|||||||||colspan="10" | ''Lesneven/Kerlouan'', [[Y. M. (04/2016)]]
|}
"""

dialects = {"léonard" : [], "cornouaillais" : [], "trégorrois": [], "vannetais" : [], "standard" : [], "inconnu" : [], "breton central" : []}
references = {} # keeps track of the references of the sentences: ref:[sentence]
sentences = {} # keeps the actual texts
dic = {} # token to lemma, pos, gloss
errorout = open('breton.errors.txt','w', encoding='utf-8')

#document contenant la liste des mots qui n'ont pas de POS
nopos = open('no_pos.txt','w', encoding='utf-8')

currentPage=''

unknownPOS = []

### EXPRESSIONS REGULIERES
repretty = re.compile(r'\{\|.*?class="prettytable"((\n|.)*?)\|\}', re.MULTILINE)
rePOS = re.compile(r'verbes|auxiliaires|copules|adverbes|complémenteurs|conjonctions|prépositions|adjectifs|noms|particules verbales|interjections|postpositions|déterminants|quantifieurs|pronoms|noms propres|suffixe', re.IGNORECASE)
reDialect = re.compile(r'léonard|cornouaillais|trégorrois|vannetais|breton central|standard', re.IGNORECASE)
reLanguage = re.compile(r'gallois|roumain|italien|espagnol|hébreu|arabe|français|anglais|allemand|moyen breton|vieux breton|breton pré-moderne', re.IGNORECASE)
reType = re.compile(r'ouvrages de recherche|références de corpus|élicitations|ouvrages pédagogiques|dictionnaires|grammaires', re.IGNORECASE)
reNoms = re.compile(r'Yann|Marijo|Anna|Angela|Alejandro|Odile|Anastazi|Nolwenn|Solange|Bastien|Soazig|Marie|Marsel|Lukaz|Lenaig|Kristof|Julie|Jenovefa|Simone|Mona|Mari', re.IGNORECASE)
relinks = re.compile(r"\[\[(.*?)\]\]")
relinks2 = re.compile(r"\[\[.*?\]\]")  
reAlternatives = re.compile(r'\([^\W\d_].* */ *[^\W\d_].*\)', re.U)
reWord = re.compile(r'\[\[[a-z]+')
nospace = ["numéraux cardinaux","nom propre", "pronom incorporé", "nom de titre", '(', "-,", 'pronom réfléchi', "nom nu", ", -", "pluriel interne", "Les pronoms d'@incise contrastifs", "particule o"]  
categories = {"V": "verbe", "Adj": "adjectif", "PL": "suffixe"}

#print(pages['Konduiñ'])
#for key in pages:
	#if key.startswith('A'):
		#print(key)

# Recherche du pos dans une page titre
# la fontion cherche le pos dans toutes les pages selon le lemme donné et retourne le pos associé
def getFirstPos(title, currentPage):
	if not title: return '?'
	mpos='' 
	if title.isupper() and title != 'R':
		mpos = title   
	if title == '[[R]]' :   
		mpos='particule_verbale'
	if title == 'P.e' or 'P/e':
		mpos='préposition' 
	if re.match(r".*\((.*)\)", title):                    
		test_pos = re.match(r".*\((.*)\)", title).group(1)                   
		if test_pos.replace(".", "") in categories:                       
			mpos = categories[test_pos.replace(".", "")]
		else: 
			mpos = test_pos.replace('.', '')            
 
	else:        
	#si le titre contient un tiret bas, on le remplace par un espace pour le trouver dans les titres de pages
		if '_' in title: 
			title = title.replace("_", " ")
		if '#' in title: 
			title = title.replace("#", "-")            
#sinon on met une majuscule à la première lettre du titre
		else:             
			title = title[0].upper()+title[1:]
	#si le titre est contenu dans le dictionnaire pages
		if title in pages:           
			wikicode=''
			#gérer les redirections (parfois les exemples se trouvent sur d'autres pages)
			if '#REDIRECTION' in pages[title]:             
				#on suit la redirection
				newtitle = relinks.search(pages[title]).group(1)                
				newtitle = newtitle[0].upper()+newtitle[1:]
				if '_' in newtitle: 
					newtitle = newtitle.replace("_", " ")            
				#et on regarde à nouveau si le titre est dans le dictionnaire pages
				if newtitle in pages:
					wikicode = pages[newtitle]              
				else: 
				# print(8888,newtitle, newtitle in pages)
					newtitle = newtitle[0].upper()+newtitle[1:].replace(' ','_')
					if newtitle in pages:
						wikicode = pages[newtitle]
					else:
						newtitle = newtitle.split(',')[0]
						newtitle = newtitle[0].upper()+newtitle[1:]
						if newtitle in pages:
							wikicode = pages[newtitle]
						else:
							errorout.write('\n'+currentPage+': strange redirect to page that does not exist: '+newtitle+'\n')
			else:
				wikicode = pages[title]                               
			if re.findall(r"\[\[Category:.*\|", wikicode):
				cats = re.findall(r"\[\[Category:.*\|", wikicode)
				list_cats = set()
				for cat in cats:
					if rePOS.search(cat) and rePOS.search(cat).group(0) not in list_cats:
						list_cats.add(rePOS.search(cat).group(0).rstrip('s'))
						if len(list_cats) == 1:                                          
							mpos = list(list_cats)[0]
						elif len(list_cats) >0: 
							mpos = "|".join(list_cats)                            
			if mpos:
				mpos = mpos
			else:
				mpos = "?"
		else:          
			if reNoms.search(title):             
				mpos = "nom_propre"
			elif "%" in title:             
				wikicode = pages[currentPage]                               
				if re.findall(r"\[\[Category:.*\|", wikicode):
					cats = re.findall(r"\[\[Category:.*\|", wikicode)
					list_cats = set()
					for cat in cats:
						if rePOS.search(cat) and rePOS.search(cat).group(0) not in list_cats:
							list_cats.add(rePOS.search(cat).group(0).rstrip('s'))
							if len(list_cats) == 1:                                          
								mpos = list(list_cats)[0]
							elif len(list_cats) >0: 
								mpos = "|".join(list_cats)                               
			else:               
				mpos = "?"                   
	return mpos


# Suppression des espaces dans les [[ ]] sinon, le split avec comme délimiteur espace va mal se passer...
# case : [[koll|perd]].[[-et (Adj.)|u]]
## ATTENTION SI DEUX ESPACES DANS UN MEME [[ ]] n'enlèvera que le premier, à améliorer !!
def sansespace(ch):
	for e in relinks.finditer(ch):
		if " " in e.group(0):
			start, end = e.start(), e.end()
			newch = re.sub(r"\[\[(.*?) (.*?)\]\]", "[[\\1_\\2]]", e.group(0))
			newch = newch.replace(' ', '_') 
			ch = ch[:start] + newch + ch[start+len(newch)+1:]
	return ch
#exemple :
#print(sansespace("[[art|le]] _[[1]]_[[hey j]]_[[maouez | femme]]"))
#resultat : "[[art|le]] _[[1]]_[[heyj]]_[[maouez| femme]]")
#print(sansespace("[[R]] [[+C]] [[ez eus|est]]"))

##  _[[1]]_exemple pas pris en compte
mutation1 = re.compile(r"_\[\[([12345])\]\]_\[\[(.*)\]\]")
mutation2 = re.compile(r"\[?\[?(.*)\]?\]?_\[\[([12345])\]\]_")

#ex: "[[ne]]_[[+C]]_"
epenthetique1 = re.compile(r"\[\[(.*)\]\]_(\[?\[?\+C\]?\]?)_")
#ex: [[ma|que]]_[[4]],+C_
epenthetique2 = re.compile(r"\[\[(.*)\]\]_\[\[([12345])\]\],(\[?\[?\+C\]?\]?)_")
#ex: [[R]]_[[+C]],[[4]]_  
epenthetique3 = re.compile(r"\[\[(.*)\]\]_(\[?\[?\+C\]?\]?),\[\[([12345])\]\]_")
#ex : [[R]]_[[+C]]_[[ez eus|est]]
#epenthetique4 = re.compile(r"\[\[(.*)\]\]_(\[?\[?\+C\]?\]?)_[\[(.*)\]\]")
#test = "_[[1]]_[[kozh|vieille]]"
#test = "([[R]]_[[1]]_)"
#print(mutation2.search(test))  
mutation = False
epenthetique = False

groupes_pipe = re.compile(r"(.*?)\|(.*)")
# fonction qui prend un token, avec ou sans double crochets [[ ]]
# NB : tr = traduction ; t = token

def tokentrans2lemposglossmorph(tr, t, title, currentPage):
	print(f't:{t}, tr:{tr}')         
	# Rq : morph a par défaut '_'
	lem, pos, gloss, morph = '', '', '', ''
	m = relinks.search(tr)     
	test_mutation1 = mutation1.search(tr)
	test_mutation2 = mutation2.search(tr)
	test_epenthetique1 = epenthetique1.search(tr)  
	test_epenthetique2 = epenthetique2.search(tr)  
	test_epenthetique3 = epenthetique3.search(tr)
    
    
    # Si c'est une mutation
	if test_mutation1 : 
		print("yes1")        
		mutation = True        
		if ('.[[' or ']].') in tr:
			tr1 = "[[" + test_mutation1.group(2) + "]]"          
			lems, glosss, poss = [], [], []
			for mbis in relinks2.finditer(tr1):                  
				try:                
					lem, pos, gloss, morph = tokentrans2lemposglossmorph(mbis.group(0), t, title, currentPage)
					lems += [lem]
					glosss += [gloss]
					poss += [getFirstPos(lem, currentPage)]  
					morph = morph + test_mutation1.group(1) + " " + "target"                    
				except:
					with open('abc.txt','w', encoding="utf-8") as f :                
						f.write(f"PROBLEMEEEEEEEEEEEEE : {tr} et {t}") 
				lem = '.'.join(lems)
				gloss = '.'.join(glosss)
				pos = '.'.join(poss)
			return lem, pos, gloss, morph
        
		else:          
			morph = test_mutation1.group(1) + " " + "target"
			token_mute = test_mutation1.group(2)       
			token_mute_pipe = groupes_pipe.search(token_mute)
			if token_mute_pipe: 
				lem = token_mute_pipe.group(1)
				gloss = token_mute_pipe.group(2)
				if gloss == 'R':  
					pos = "particule_verbale" 
				else:   
					pos = getFirstPos(lem, currentPage)
			else :
				lem = t        
				gloss = token_mute   
				if gloss == 'R'  or '[[R]' in gloss:  
					pos = "particule_verbale" 
				else:                          
					pos = getFirstPos(t, currentPage)
				if pos == '?':
					pos = getFirstPos(tr, currentPage)
                    
                    
	elif test_mutation2 :
		print("yes2")  
		mutation = True        
		if ('.[[' or ']].') in tr:                
			tr1 = "[[" + test_mutation2.group(2) + "]]"               
			lems, glosss, poss = [], [], []
			for mbis in relinks2.finditer(tr1):            
				try:                
					lem, pos, gloss, morph = tokentrans2lemposglossmorph(mbis.group(0), t, title, currentPage)
					lems += [lem]
					glosss += [gloss]
					poss += [getFirstPos(lem, currentPage)]
					morph = morph + test_mutation2.group(2) + " " + "trigger"
				except:
					with open('abc.txt','w', encoding="utf-8") as f :                
						f.write(f"PROBLEMEEEEEEEEEEEEE : {tr} et {t}") 
				lem = '.'.join(lems)
				gloss = '.'.join(glosss)
				pos = '.'.join(poss)
			return lem, pos, gloss, morph
        
		else:              
			morph = test_mutation2.group(2) + " " + "trigger"
			token_mute = test_mutation2.group(1)       
			token_mute_pipe = groupes_pipe.search(token_mute)          
			if token_mute_pipe:      
				lem = token_mute_pipe.group(1)
				gloss = token_mute_pipe.group(2)
				if gloss == 'R'  or '[[R]' in gloss:  
					pos = "particule_verbale" 
				else:                
					pos = getFirstPos(lem, currentPage)
			else :
				lem = t
				gloss = token_mute
				if gloss == 'R'  or '[[R]' in gloss:  
					pos = "particule_verbale"            
				else:
					pos = getFirstPos(t, currentPage)
				if pos == '?':
					pos = getFirstPos(tr, currentPage)
#S'il y a une consonne épenthétique (+C)             
	elif test_epenthetique1 : 
		epenthetique = True    
		#print("yes3") 
		morph = test_epenthetique1.group(2) 
		token_mute = test_epenthetique1.group(1)       
		token_mute_pipe = groupes_pipe.search(token_mute)
		if token_mute_pipe:      
			lem = token_mute_pipe.group(1)
			gloss = token_mute_pipe.group(2)
			if gloss == 'R' or '[[R]' in gloss:  
				pos = "particule_verbale" 
			else:                
				pos = getFirstPos(lem, currentPage)
		else :
			# print('just a word')
			lem = t
			gloss = token_mute
			if gloss == 'R' or '[[R]' in gloss:  
				pos = "particule_verbale"                  
			else:
				pos = getFirstPos(t, currentPage)
			if pos == '?':
				pos = getFirstPos(tr, currentPage)

	elif test_epenthetique2 : 
		epenthetique = True    
		morph = test_epenthetique2.group(2) + '|' +  test_epenthetique2.group(3) 
		token_mute = test_epenthetique2.group(1)    
		token_mute_pipe = groupes_pipe.search(token_mute)
		if token_mute_pipe:      
			lem = token_mute_pipe.group(1)
			gloss = token_mute_pipe.group(2)
			if gloss == 'R' or '[[R]' in gloss:  
				pos = "particule_verbale" 
			else:                
				pos = getFirstPos(lem, currentPage)
		else :
			# print('just a word')
			lem = t
			gloss = token_mute
			if gloss == 'R' or '[[R]' in gloss:  
				pos = "particule_verbale"                  
			else:
				pos = getFirstPos(t, currentPage)
			if pos == '?':
				pos = getFirstPos(tr, currentPage)
                             
	elif test_epenthetique3:
		epenthetique = True    
		morph = test_epenthetique3.group(2) + '|' +  test_epenthetique3.group(3)
		token_mute = test_epenthetique3.group(1)       
		token_mute_pipe = groupes_pipe.search(token_mute)
		if token_mute_pipe:      
			lem = token_mute_pipe.group(1)
			gloss = token_mute_pipe.group(2)
			if gloss == 'R' or '[[R]' in gloss:  
				pos = "particule_verbale" 
			else:                
				pos = getFirstPos(lem, currentPage)
		else :
			lem = t
			gloss = token_mute
			if gloss == 'R' or '[[R]' in gloss:  
				pos = "particule_verbale"                  
			else:
				pos = getFirstPos(t, currentPage)
			if pos == '?':
				pos = getFirstPos(tr, currentPage)                    
                               
# si ce n'est pas une mutation et si on a des double crochets [[ ]]              
	elif m:        
		# cas "amalgames" et/ou mots contenant un tiret: exemple [[da|à]].[[pronom incorporé|vous]] ou [[den|parent]].[[pluriel interne|s]]-[[kozh|vieil]]  
		if ('.[[' or ']].' or '-[[' or ']]-' ) in tr:      
			lems, glosss, poss, ponct = [], [], [], [] 
			lemt, glosst, post = [], [], []
			if '(Adj.)' or '(V.)' or '(PL.)' or 'P.e' or '(F.)' in tr:      
				tr = tr.replace('(Adj.)', '(Adj)').replace('(V.)', '(V)').replace('(PL.)', '(PL)').replace('P.e', 'P/e').replace('(F.)', '(F)')   
			#s'il y a des tirets dans les groupes entre crochets, on les remplace par le signe #        
			#ex: [[genel|nat]].[[-idig|if]]  -->  [[genel|nat]].[[#idig|if]]             
			if re.search(r"\[ ?- ?[A-Za-z]", tr):               
				suffixe = re.search(r"\[ ?(-) ?[A-Za-z]", tr).group(1)
				tr = re.sub(r'(?<=\[) ?- ?(?=[A-Za-z])', '#', tr )
			tr = tr.replace(".", " .").replace("-", " -")                    
			for i in range(len(tr.split())):         
				test = tr.split()[i]    
				if relinks2.search(test):                 
					ele = relinks2.search(test).group(0)                                        
					lem, pos, gloss, morph = tokentrans2lemposglossmorph(ele, t, title, currentPage)   
					lems += [lem]
					glosss += [gloss]
					poss += [getFirstPos(lem, currentPage)]
					# except:
					# 	print("error")                        
					# 	with open('abc.txt','w', encoding="utf-8") as f :                
					# 		f.write(f"PROBLEMEEEEEEEEEEEEE : {tr} et {t}") 
				else:                                  
					try:                    
						lem, pos, gloss, morph = tokentrans2lemposglossmorph(test, t, title, currentPage)                 
						lems += [lem]
						glosss += [gloss]
						poss += [getFirstPos(lem, currentPage)]
					except:
						with open('abc.txt','w', encoding="utf-8") as f :                
								f.write(f"PROBLEMEEEEEEEEEEEEE : {tr} et {t}")  
				#on "recolle" les éléments avec le bon séparateur (tiret ou point)                                
				if tr.split()[i].startswith("."): 
					#print(f"point : {tr.split()[i]}, {lems[i]}, {glosss[i]}, {poss[i]}")
					lemt.append(f".{lems[i].replace('#', '-')}")
					glosst.append(f".{glosss[i]}")  
					post.append(f".{poss[i]}")  
				elif tr.split()[i].startswith("-"): 
					#print(f"tiret : {tr.split()[i]}, {lems[i]}, {glosss[i]}, {poss[i]}")
					lemt.append(f"-{lems[i].replace('#', '-')}")
					glosst.append(f"-{glosss[i]}")  
					post.append(f"-{poss[i]}")  
				else: 
					#print(f"rien : {tr.split()[i]}, {lems[i]}, {glosss[i]}, {poss[i]}") 
					lemt.append(lems[i].replace('#', '-')) 
					glosst.append(glosss[i])                    
					post.append(poss[i])                                        
                        
            
			# lem = '-'.join(lems)
			# gloss = '-'.join(glosss)
			# pos = '-'.join(poss)
            
			lem = ''.join(lemt)
			gloss = ''.join(glosst)
			pos = ''.join(post)             
        
			#print('__lem, pos, gloss:', lem, pos, gloss)
			return lem, pos, gloss, morph        
        
        
        
		# cas "normal" : pas d'amalgame, un token et une gloss --> [[token|gloss]]
		if '|' in m.group(1):      
			if "numéraux_cardinaux" in m.group(1): 
				lem = t
				pos = m.group(1).split('|')[0]
				gloss = m.group(1).split('|')[1] 
			else:                 
				lem = m.group(1).split('|')[0]
				gloss = m.group(1).split('|')[1]                  
				if gloss == 'R' or '[[R]' in gloss:  
					pos = "particule_verbale"                      
				elif lem == 'P.e':  
					lem = "e"
					pos = "préposition"               
				elif lem == 'nom_propre':  
					pos = lem
					lem = gloss                 
				else:
					pos = getFirstPos(lem, currentPage)                        
				if pos=='?':
				# ?
					pos = getFirstPos(t, currentPage)
                
		# cas sans amalgame mais sans gloss --> [[ne]]
		else:            
			lem = t
			gloss = relinks.search(tr).group(1)
			if gloss == 'R'  or '[[R]' in gloss:  
				pos = "particule_verbale"
			if 'DIM' in gloss and  t.endswith('ig') or t.endswith('ig%'):
				lem = "-ig"                
				pos = "suffixe"                
			else:
				pos = getFirstPos(t, currentPage)
			if pos == '?':
				pos = getFirstPos(tr, currentPage)      
	# cas d'un mot seul sans crochets --> foot dans la page "Abardaez,_enderv"      
	else :
#		print('just a word')
		lem = t
		gloss = tr
		if gloss == 'R'  or '[[R]' in gloss:  
			pos = "particule_verbale"            
		else:
			pos = getFirstPos(t, currentPage)
			if pos == '?':
				pos = getFirstPos(tr, currentPage)
	#print('__lem, pos, gloss:', lem, pos, gloss)
	return lem, pos, gloss, morph


# Nettoyage des tokens
def cleanToken(t):
    
	## ATTENTION ''' ''' veut dire que le pos du token est sur la page même ! peut être à utiliser..      
	t = t.replace("'''", "%")
	t = t.replace("''", "")
	t = re.sub(r'</?font.*?>','',t)
	t = re.sub(r'<sup>','_',t)
	t = re.sub(r'</sup>','_',t)
	t = re.sub(r'<u>','',t)
	t = re.sub(r'</u>','',t)
	t = re.sub(r'<sub>.*?</sub>','',t)
	t = re.sub(r'\(\[\[\*\]\].*?\)','$$$',t)
	t = re.sub(r"c'h",'cxxxh',t)
	t = re.sub(r"C'h",'Cxxxh',t)
	t = re.sub(r"`",'',t) 
	t = re.sub(r"#",'-',t)     
	## ne pas splitter quand l'apostrophe est en début de mot (Rannig) -> pas réussi avec la regex    
	#if t.startswith("'"): 
		#print(f"it's heere {t}")        
		#t = re.sub(r"'([^ ])","%\\1",t)
        
	# pour spliter sur les apostrophes sans les enlever : mettre @ pour spliter sur @ et donc conserver l'apostrophe
	##ATTENTION guillemet en début de mot puis token collés mal fait 
	t = re.sub(r"'","'@",t)
	return t.strip()

#test
#print(cleanToken("'anava"))

# Remettre les tokens qui ont été remplacés
def remettretoken(t):
	t = re.sub(r"%","",t)
	t = re.sub(r'cxxxh',"c'h",t)
	t = re.sub(r'Cxxxh',"C'h",t)
	#t = re.sub(r"%","'",t)
	t = re.sub(r"'@", "'",t)
	t = re.sub(r"\$\$\$ ", "",t)
	return t

# Savoir si une string contient un nombre
def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)

def analyzePage(title): 
	currentPage = title
	mpos = rePOS.search(pages[title])
	pagepos = None
	conlls=[]
	grammatical = True
	s= ''
	if mpos:
		# mets tout en minuscule
		pagepos = mpos.group(0).lower()
		# print('pagepos',pagepos)
	#pour chaque tableau (et donc chaque exemple)
	nb_tables = 0    
	for m in repretty.finditer(pages[title]):
		nb_tables += 1        
		# print('\n\n_______________',m.group(1))
		tokens, trans = [], []
		translation, source, source2 = '', '', ''
		location, dialect, phonetic, texttype, l = '','','', '', ''
		grammatical=True         
		# si le tableau est "standard" avec une ligne de token, une ligne de glose et une ligne source
		if (len(m.group(1).split('\n'))) == 9 :
			for li in m.group(1).split('\n'): # for every line of the table
				if li[:2]=='|(' :                  
					if '[[*]]' in li.split('||')[0]:
						grammatical = False                       
					tokens = [cleanToken(t) for t in li.split('||')[1:]]   
					if tokens[0]=="[[*]]" :
						grammatical = False
						break      
					if '[' in tokens[0]:
						tokens[0] = tokens[0].replace('[', '').strip()
						#phonetic[0] = phonetic[0].replace('[', '')         
					if ']' in tokens[-1]: 
						tokens[-1] = tokens[-1].replace(']', '').strip()
						#phonetic[-1] = phonetic[-1].replace(']', '')
					#MODIF !!!                                       
                        
				if li[:3]=='|||' and grammatical:
					if not trans: #la première fois = la traduction                      
						trans = [cleanToken(tr) for tr in li[3:].split('||')]  
						#print(f"là : {trans}")                    
						for i in range(len(trans)):
							if any(ele in trans[i] for ele in nospace):                               
								trans[i] = trans[i].replace(" ", "_")                                
							if "jours de la semaine" in trans[i]:
								trans[i] = sansespace(trans[i]) 
							#voir comment faire car peut causer des problèmes: 
							#ex: [[R]]_[[1]]_ [[COP|sera]]                            
							# if "_ " in trans[i]:
							# 	trans[i] = trans[i].replace("_ ", "_")
						#print(f"après : {trans}")                                
						# print(len(tokens), len(tokens)==len(trans))
					elif 'colspan=' in li:                  
						# vu que les deux lignes translation et source remplissent cette condition, quand on arrive à source
						# la string translation est remplie donc on n'écrase pas et on passe à source
						if not translation:
							translation = li.split('|')[-1]
							for i in range(len(trans)): 
								if any(ele in trans[i] for ele in nospace):
									trans[i] = trans[i].replace(" ", "_")     
								if "jours de la semaine" in trans[i]:
									trans[i] = sansespace(trans[i]) 
							# if "_ " in trans[i]:
							# 	trans[i] = trans[i].replace("_ ", "_")
                                
						elif not source: # si déjà trans, alors on a la source
							source = li.split('|')[-1].replace('[[','').replace(']]','').replace("''",'').replace(' ','')                           
							if reLanguage.search(source):                              
								grammatical = False     
								break                                
							if re.findall("''.*''", li):                 
								if re.search(r"\)''", li):           
									location = re.findall(r"\(([^\)]+)\)", li)[0]
									if has_numbers(location):
										location = ''
							if reDialect.search(li): 
								dialect = reDialect.search(li)[0]
							if re.search(r"\[\[(.*?)\]\]", li):                                     
								l = re.search(r"\[\[(.*?)\]\]", li).group(0) 
								lang = re.search(r"(.*?)\[\[", li).group(0)
								if reLanguage.search(lang): 
									grammatical = False                                   
								l = str(l).split('|')[0].replace('[[', '').replace(']]', '').replace("'", "")
								if l in pages:                                 
									if '#REDIRECTION' in pages[l]:                      
										#on suit la redirection                                        
										newtitle = relinks.search(pages[l]).group(1)
										newtitle = newtitle[0].upper()+newtitle[1:]
										s = newtitle
										if newtitle in pages:
											bib_source = pages[newtitle]
											if "moyen breton" in str(re.findall(r"\[\[Category:.*\|", bib_source)):
												s += '(moyen breton)'                                           
									else: 
										s = l                                    
										bib_source = pages[l]
									if re.findall(r"\[\[Category:.*\|", bib_source):
										bibl = re.findall(r"\[\[Category:.*\|", bib_source)                   
										for ele in bibl:
											tdialect = reDialect.search(ele)
											if tdialect:
												dialect = tdialect.group(0).capitalize().replace("[[Category:", "").replace("|", "")
											ttype = reType.search(ele)
											if ttype:
												texttype = ttype.group(0).capitalize().replace("[[Category:", "").replace("|", "") 
										if not dialect: 
											if reDialect.search(source):
												dialect = reDialect.search(source).group(0)                               
								else: 
									s = l
							if not location:
								if re.search("(''.*'')", li):                                
									test = re.search("''(.*)''", li).group(1) 
									if not reDialect.search(test) and not test.endswith("ais"):                                     
										location = test                                   
							# print(111,source)
			# print(tokens, trans, translation, source)
			if not( tokens and trans and translation and source): # if one is missing
				#if verbose: print('Houston, we got a problem: The number of tokens and the number of transcriptions differ, or translation or source are missing:', title) 
				errorout.write('\n'+currentPage+': The number of tokens and the number of transcriptions differ or translation or source are missing:'+m.group(1)+'\n')
				continue
		# S'il y a plus de lignes que prévu : par ex une ligne de phonétique ou deux lignes source, ou les deux
		else:
			for li in m.group(1).split('\n'): # for every line of the table
				#si ce n'est pas une ligne de phonétique                
				if li[:2]=='|(' and bool(re.search(r'</?font.*?>',li))==False:
					tokens = [cleanToken(t) for t in li.split('||')[1:]]
						#tokens[0] = tokens[0].replace('[', '')         
					if ']' in tokens[-1]: 
						tokens[-1] = tokens[-1].replace(']', '')
                        
				#si c'est une ligne de phonétique
				if li[:2]=='|(' and bool(re.search(r'</?font.*?>',li))==True:
					phonetic = [cleanToken(t) for t in li.split('||')[1:]]
					clean_phonetic = ''
					if '[' or '/' in phonetic[0]: 
						phonetic[0] = phonetic[0].replace('[', '').replace('/', '')         
					if ']' or '/' in phonetic[-1]: 
						phonetic[-1] = phonetic[-1].replace(']', '').replace('/', '')
					for ele in phonetic:        
						clean_phonetic += ele
						clean_phonetic += ' '

				if li[:3]=='|||':
					# si on a rencontré une ligne phonétique et donc qu'on a pas encore de tokens, cette ligne (2e ligne) est la ligne de tokens                   
					if not tokens :
						tokens = [cleanToken(t) for t in li[3:].split('||')]                
						#print(tokens)
					elif not trans: #la première fois = la traduction
						trans = [cleanToken(tr) for tr in li[3:].split('||')]
						#print(f"ici trans {trans}")                        
						for i in range(len(trans)):                            
							if any(ele in trans[i] for ele in nospace):
								trans[i] = trans[i].replace(" ", "_")     
							if "jours de la semaine" in trans[i]:
								trans[i] = sansespace(trans[i]) 
							if "_ " in trans[i]:
								trans[i] = trans[i].replace("_ ", "_")
					elif 'colspan=' in li:
						if li.startswith('|||colspan='):
							translation = li.split('|')[-1]
						elif not source:                        
							source = li.split('|')[-1].replace('[[','').replace(']]','').replace("''",'')   
							if reLanguage.search(li):
								grammatical = False                             
						elif not source2:                            
							source2 = li.split('|')[-1].replace('[[','').replace(']]','').replace("''",'')    
							if reLanguage.search(li):
								grammatical = False 
						if source:             
							if source2 : 
								source = source2                                
							#source, source2 = source2, source
							if re.findall("''.*''", li):                 
								if re.search(r"\)''", li):           
									location = re.findall(r"\(([^\)]+)\)", li)[0]
									if has_numbers(location):
										location = ''
							if reDialect.search(li): 
								dialect = reDialect.search(li)[0]
							if re.search(r"\[\[(.*?)\]\]", li):                                
								l = re.search(r"\[\[(.*?)\]\]", li).group(0)                              
								lang = re.search(r"(.*?)\[\[", li).group(0)
								if reLanguage.search(lang): 
									grammatical = False                                   
								l = str(l).split('|')[0].replace('[[', '').replace(']]', '').replace("'", "")                          
								if l in pages:                                 
									if '#REDIRECTION' in pages[l]:                                       
										#on suit la redirection
										newtitle = relinks.search(pages[l]).group(1)                
										newtitle = newtitle[0].upper()+newtitle[1:]
										s = newtitle
										if newtitle in pages:
											bib_source = pages[newtitle]
											if "moyen breton" in str(re.findall(r"\[\[Category:.*\|", bib_source)):
												s += '(moyen breton)'                                           
									else: 
										s = l                                              
										bib_source = pages[l]
									if re.findall(r"\[\[Category:.*\|", bib_source):
										bibl = re.findall(r"\[\[Category:.*\|", bib_source)                 
										for ele in bibl:
											tdialect = reDialect.search(ele)
											if tdialect:
												dialect = tdialect.group(0).capitalize().replace("[[Category:", "").replace("|", "")
											ttype = reType.search(ele)
											if ttype:
												texttype = ttype.group(0).capitalize().replace("[[Category:", "").replace("|", "") 
										if not dialect: 
											if reDialect.search(source):
												dialect = reDialect.search(source).group(0)
								if reLanguage.search(source):
									grammatical = False                                                 
								else: 
									s = l
							if not location:
								if re.search("(''.*'')", li):                                
									test = re.search("''(.*)''", li).group(1) 
									if not reDialect.search(test) and not test.endswith("ais"):                                     
										location = test 
                           
			# print(tokens, trans, translation, source)
			if not( tokens and trans and translation and source): # if one is missing
				#if verbose: print('Houston, we got a problem: The number of tokens and the number of transcriptions differ, or translation or source are missing:', title) 
				errorout.write('\n'+currentPage+': The number of tokens and the number of transcriptions differ or translation or source are missing:'+m.group(1)+'\n')
				continue            
	#pour supprimer les mots agrammaticaux 
	#exemple: ( C'hwi / * Ac'hanoc'h ) --> C'hwi  
		cl_tokens = []    
		for i in range(len(tokens)-1):         
			if '[...]' in tokens[i]:
				tokens[i] = tokens[i].replace('[...]', '')
			if '(...)' in tokens[i]:
				tokens[i] = tokens[i].replace('(...)', '')              
			if re.match(r".*\((.*\.)\)", title):            
				ele = re.match(r".*\((.*)\.\)", title).group(1)                
				tokens[i] = tokens[i].replace(ele, "")                
			if '/' in tokens[i] and '*' in tokens[i]:                
				elems = tokens[i].split('/')                
				for m in range(len(elems)):                    
					idx = m                    
					if '*' in elems[m]:                        
						tokens[i] = tokens[i].replace(elems[m], '').replace('(', '').replace('{', '').replace('/', '')   
						cl_tokens.append(tokens[i])                        
						for i in range(len(trans)) :              
							if '/' in trans[i]:                       
								trans[i] = trans[i].split('/')[idx].replace('(', '').replace('{', '').replace('/', '')
			elif '/' in tokens[i] and not '*' in tokens[i]: 
				#print(tokens)             
				if reAlternatives.search(tokens[i]):  
					tokens[i] = tokens[i].split('/')[0].replace('(', '').replace('{', '').replace('/', '')
					trans[i] = trans[i].split('/')[0].replace('(', '').replace('{', '').replace('/', '')   
					#print(tokens[i])
                                    
		if 'Équivalent standardisé' in tokens: 
			tokens.remove('Équivalent standardisé')
		#supprime les éléments vides de la liste) 
  
		#print(f"encore après {trans}")              
		tokens = ' '.join(tokens).split()      
		trans = ' '.join(trans).split()    
		#print(tokens, trans)
		if grammatical is False: 
			break
        
		# construction of the conll:  
		text = ' '.join(tokens)
		text_ch = remettretoken(text) 


		# construction of the conll:         
		text = ' '.join(tokens)
		text_ch = remettretoken(text)        
		# print(text, text_ch)
		references[source] = references.get(source,[])
		if text in references[source]:
			ind = references[source].index(text)+1
		else:
			references[source]+=[text]
			ind = 1
		sentences[text] = sentences.get(text,0)+1
		conllis = ['# sent_id = '+source.replace(' ', '') +'__'+str(ind)]
		conllis += ['# text = '+text_ch]
		#MODIF !!! 
		conllis += ['# text_fr = '+ remettretoken(cleanToken(translation)).replace('[...]', '').replace('(...)', '')]
		if phonetic:
			conllis += ['# text_phon = '+clean_phonetic.replace('[...]', '').replace('(...)', '')]
		if dialect:
			conllis += ['# dialect = ' + dialect]
		if location:
			conllis += ['# location = ' + location]
		if source2:
			conllis += ['# source2 = ' + source2]
		if s :
			conllis += ['# source = ' + s] 
		if texttype :
			conllis += ['# texttype = ' + texttype]   
		##si on a trouvé du vert alors conllis += ['# phonetic = yes ']
		##si un morceau en phonétique alors mettre en trait MISC

		extrai = 1
		# si un token en trop dans les tokens, alors enlever le dernier 
		# case : ||| N'eo || ket || dereat || teañ || e || dad. || ''Équivalent standardisé''
		# if len(tokens) == len(trans)+1:
		# 	tokens = tokens[:-1] 
   
		for i,t in enumerate(tokens):       
			if not t:
				errorout.write('\n___ '+currentPage+': strange token nr '+str(i)+' in ')
				t='???'
			if len(trans)<=i:
				errorout.write('\n___ '+currentPage+': strange token nr '+str(i)+' in ')
				tr = '???'
			else:
				tr = trans[i]
                
			# délimiteur : @ et espace 
			# ATTENTION : tiret en délimiteur non traité pour l'instant !! (faire la même chose que la fonction sansespace, il faut splitter)
			# sur les tirets EN DEHORS des doubles crochets, non ceux qui sont à l'intérieur          
			#print(f"t {t, tr}")
			#print(tr)            
			test1 = re.split("@| ", t)
			if '' in test1:  
				test1.remove("")         
			test2 = sansespace(tr).split(" ") 
			#print(test1)
			# si on a le même nombre d'éléments pour les tokens et la glose entre chaque || ||
			if len(test1) == len(test2):               
				for j,e in enumerate(test1):
					t1 = test1[j]                  
					tr1 = test2[j]  
#					print(f"ici {t1, tr1}")                    
					try:                    
						if t1[-1] not in ".,;!?" and t1 != "$$$":  
							print("1")                
							lem, pos, gloss, morph = tokentrans2lemposglossmorph(tr1, t1, title, currentPage)                         
							dic[t1] = dic.get(t1, [])+[(lem,pos,gloss)]# et morph ?
							eles = [str(extrai),remettretoken(t1), remettretoken(lem), pos]+5*['_']+['Gloss='+ remettretoken(gloss)]
							if morph != '':
								eles[9]+='|Mutation='+morph.split(" ")[1]+'|MutationNUM='+morph.split(" ")[0]
							if len(eles) == 10:                              
								conllis += ['\t'.join(eles)]
							else: 
								print(f"error: ligne du conll invalide (longueur {len(eles)})")                    
							extrai += 1
                        
						if t1[-1] in ".,;!?" and not t1[-3:] == "..." :
							print("2") 
							lem, pos, gloss, morph = tokentrans2lemposglossmorph(tr1, t1[:-1], title, currentPage)
							#si on ne trouve pas de pos, on cherche dans la page elle-même
							if pos=='?' and t1[:-1].lower()==title.lower() and pagepos:
								# couldn't find a pos, maybe it's the word of the page itself?
								pos = pagepos
							dic[t1[:-1]] = dic.get(t1[:-1], [])+[(lem,pos,gloss)]
							eles = [str(extrai),remettretoken(t1[:-1]), remettretoken(lem), pos]+5*['_']+['Gloss='+ remettretoken(gloss)]
							if morph != '':
								eles[9]+='|Mutation='+morph.split(" ")[1]+'|MutationNUM='+morph.split(" ")[0]          
							conllis += ['\t'.join(eles)]
							extrai += 1
							dic[t1[-1:]] = dic.get(t1[-1:], [(t1[-1:], 'PUNCT','punct')])
							eles = [str(extrai),remettretoken(t1[-1:]), remettretoken(t1[-1:]), 'PUNCT']+5*['_']+['Gloss=punct']
							if len(eles) == 10:                              
								conllis += ['\t'.join(eles)]
							else: 
								print(f"error: ligne du conll invalide (longueur {len(eles)})")  
							#print('\t'.join(eles))
							extrai += 1

						if t1[-3] == "..." :         
							print("3") 
							lem, pos, gloss, morph = tokentrans2lemposglossmorph(tr1, t1[:-3], title, currentPage)
							#si on ne trouve pas de pos, on cherche dans la page elle-même
							if pos=='?' and t1[:-3].lower()==title.lower() and pagepos:
								# couldn't find a pos, maybe it's the word of the page itself?
								pos = pagepos
							dic[t1[:-3]] = dic.get(t1[:-3], [])+[(lem,pos,gloss)]
							eles = [str(extrai),remettretoken(t1[:-3]), remettretoken(lem), pos]+5*['_']+['Gloss='+ remettretoken(gloss)]
							if morph != '':
								eles[9]+='|Mutation='+morph.split(" ")[1]+'|MutationNUM='+morph.split(" ")[0]                            
							conllis += ['\t'.join(eles)]
							extrai += 3
							dic[t1[-3:]] = dic.get(t1[-3:], [(t1[-3:], 'PUNCT','punct')])
							eles = [str(extrai),remettretoken(t1[-3:]), remettretoken(t1[-3:]), 'PUNCT']+4*['_']+['Gloss=punct']    
							if len(eles) == 10:                              
								conllis += ['\t'.join(eles)]
							else: 
								print(f"error: ligne du conll invalide (longueur {len(eles)})")                             
							extrai += 3 
					except:
						continue                        
                        
			# si on a le nombre d'éléments de tokens est supérieur à celui de la glose                  
			elif len(test1)>len(test2):

				for j,e in enumerate(test1):
					t1 = test1[j]                       
					if t1 == "$$$":
						print("AAAAAA")                        
					try:                    
						tr1 = test2[j]
					except:
						tr1 = "?"
					# if t1.startswith("'") and "[[R" in tr1.split("_")[0] and reWord.search(tr1) : 
					# 	m = re.match(r'(\[\[R\]\])(.*)', tr1)
					# 	rannig = m.group(1)
					# 	reste = m.group(2)                        
					# 	tr1 = [rannig, reste]
					# 	print(tr1, type(tr1), type(tr1[0]))                        
                        
                        
					# si la ponctuation est tout simplement séparée du token                   
					if t1 in "/-.,;!?":
						print("4") 
						dic[t1] = dic.get(t1, [(t1, 'PUNCT','punct')])
						eles = [str(extrai),t1, t1, 'PUNCT']+4*['_']+['Gloss=punct']
						if len(eles) == 10:                              
							conllis += ['\t'.join(eles)]
						else: 
							print(f"error: ligne du conll invalide (longueur {len(eles)})")                        
						extrai += 1
                        
					if t1 in "...":
						print("5") 
						dic[t1] = dic.get(t1, [(t1, 'PUNCT','punct')])
						eles = [str(extrai),t1, t1, 'PUNCT']+5*['_']+['Gloss=punct']
						if len(eles) == 10:                              
							conllis += ['\t'.join(eles)]
						else: 
							print(f"error: ligne du conll invalide (longueur {len(eles)})")                  
						extrai += 1             
                    
					# les autres tokens sont traités normalement
					else:
						print("6") 
						lem, pos, gloss, morph = tokentrans2lemposglossmorph(tr1, t1, title, currentPage)
						dic[t1] = dic.get(t1, [])+[(lem,pos,gloss)]
						eles = [str(extrai),remettretoken(t1), remettretoken(lem), pos]+5*['_']+['Gloss='+ remettretoken(gloss)]                       
						if morph != '':
							try:                             
								eles[9]+='|Mutation='+morph.split(" ")[1]+'|MutationNUM='+morph.split(" ")[0]
							except: 
								print("error") 
						if len(eles) == 10:                              
							conllis += ['\t'.join(eles)]
						else: 
							print(f"error: ligne du conll invalide (longueur {len(eles)})")                            
						extrai += 1

			# si on a le nombre d'éléments de la glose est supérieur à celui des tokens 
			elif len(test1)<len(test2):
				print("PROBLEME ! éléments de la glose esupérieur à celui des tokens")                
				#errorout.write('\n___ '+currentPage+': strange token nr ' +str(i)+' in '+m.group(1) + 'test1' + str(len(test1)) + 'test2' + str(len(test2)))   
		#print(conllis)              
		if any("# dialect" in string for string in conllis):         
			for d in dialects:
				if f'# dialect = {d.capitalize()}' in conllis: 
					dialects[d] += [conllis]  
		else: 
			dialects['inconnu'] += [['\n'.join(conllis)]]             
            
	print(nb_tables) 
	return dialects

print('____________\n')

#for conll in analyzePage('Konduiñ'):
analyzePage("Memes")
for d in dialects:
	for conll in dialects[d]:   
		print('\n'.join(conll) + '\n\n')

____________

1
t:Ha, tr:[[&|et]]_[[mercxxxh|femme]].[[-ed_(PL.)|s]]
t:Ha, tr:[[&|et]]
t:Ha, tr:[[#ed_(PL)|s]]
1
t:mercxxxhed, tr:[[nom_propre|Tréboul]]
1
t:Treboull, tr:[[kaout|avait]]
1
t:noe, tr:[[ket|pas]]
1
t:ket, tr:[[tre|tt.à.fait]]
1
t:tre, tr:[[art|le]]
1
t:ar, tr:même
1
t:%memes%, tr:[[taol_(M.)|coup]]_[[dorn|main]]
1
t:taol, tr:[[da|pour]]_[[1]]_
yes2
1
t:dorn, tr:[[lakaat|mettre]]
1
t:da, tr:[[POSS|leur]]_[[2]]_
yes2
1
t:lakaat, tr:[[koef|coiffe]]
1
t:o, tr:???
2
t:cxxxhoef, tr:???
1
t:Ne, tr:[[ne]]_[[1]]_
yes2
1
t:wriont, tr:[[gwriat|cousent]]
1
t:ket, tr:[[ket|pas]]
1
t:war, tr:[[war|sur]]
1
t:ar, tr:[[art|le]]
1
t:%memes%, tr:même
2
t:liñser, tr:[[liñser|drap]]
1
t:met, tr:[[met|mais]]
1
t:nend, tr:[[ne]]_[[+C]]_
1
t:aomp, tr:[[mont|allons]]
1
t:ket, tr:[[ket|pas]]
6
t:d', tr:[[da|à]]
6
t:ar, tr:?
1
t:%memp%, tr:[[art|le]]
2
t:tu, tr:même
1
t:Hag, tr:[[&|et]]
1
t:amañ, tr:[[amañ|ici]]
2
t:%memp%, tr:même
1
t:bout, tr:[[bez'@|expl]]
6
t:', tr:[[R]]
6
t:h, tr:?
1
t:eus, tr

In [ ]:
print(dic)

In [ ]:
for conll in analyzePage('Memes'):
	print(conll,'\n')

In [ ]:
print(dic)

In [6]:
def writeDic():
	out = open('breton.dic.tsv','w', encoding="utf-8")
	for t, lpg in sorted(dic.items()):
		lpg = sorted(set(lpg))
		
		lines = ['\t'.join([t,l,p,g]) for l,p,g in lpg]
		out.write('\n'.join(lines)+'\n')
writeDic()

# trying to run through the whole site:

In [ ]:
print(len(pages))
dic = {}
errorout = open('breton.errors.txt','w', encoding="utf-8")
nopos = open('no_pos.txt', 'w', encoding='utf-8')
for title in tqdm.tqdm(pages):
	# print('____________',title)
	analyzePage(title)  
for d in dialects:
	for conll in dialects[d]:   
		open('bretonconlls/'+re.sub(r'\W','_',d)+'.conllu','a', encoding="utf-8").write('\n'.join(conll) + '\n\n')    
#	if conlls:
#		open('bretonconlls/'+re.sub(r'\W','_',title)+'.conllu','w', encoding="utf-8").write('\n\n'.join(conlls))
#writeDic()

8068


  0%|▎                                                                              | 26/8068 [00:00<00:32, 249.94it/s]

PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens


  1%|▍                                                                              | 51/8068 [00:00<00:39, 205.30it/s]

PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens


...

PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME

  6%|████▌                                                                         | 478/8068 [00:01<00:39, 190.27it/s]

PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens


  6%|████▉                                                                         | 505/8068 [00:02<00:51, 148.24it/s]

PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
PROBLEME ! éléments de la glose esupérieur à celui des tokens
